# HOUGH Detection

In this notebook, we convert the data from each tile into an image, process this image and detect the fiducial circles.

The fiducial circles are detected using a HOUGH transform and we identify 2 narrow sizes to increase accuracy where possible.

The centroids of the fiducials are used to correct for the global positions of the tiles across a Nova-ST chip.

In [ ]:
import pysam
import collections as c
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import numpy as np
import cv2 as cv

%matplotlib inline

import time
import os
import time
import datetime
from multiprocessing import Pool
import PIL

In [ ]:
from scipy.sparse import coo_array

In [ ]:
TILE_DIR = 'HDMI_Tiles_Data' # The output data folder from 1.1.HDMI_Processing

In [ ]:
def process_tile(info, return_images=False):
    tiles_dir, tile_id = info
    try:
        with open(f'{tiles_dir}/{tile_id}_barcodes.pickle', 'rb') as tile_fh:
            tile_data = pickle.load(tile_fh)
    except FileNotFoundError:
        return (f'{tile_id}', None, None)

    # Tile data structure: key: barcode, values: (x, y)
    xs = [int(x[0]) for x in tile_data.values()]
    ys = [int(x[1]) for x in tile_data.values()]
    max_x = max(xs)
    max_y = max(ys)

    # Create a matrix of of zeros in the size of the tile
    arr_orig = np.zeros((max_x + 1, max_y + 1) , dtype=np.uint8)
    for x, y in zip(xs, ys):
        arr_orig[x, y] = 255

    # Shrink the data by binning for easier processing. Lower numbers = higher resolution but more computation time
    IMG_BINSIZE = 25
    m = arr_orig.shape[0]
    n = arr_orig.shape[1]
    trim_arr = arr_orig[:(m // IMG_BINSIZE) * IMG_BINSIZE, :(n // IMG_BINSIZE) * IMG_BINSIZE]
    arr_re = trim_arr.reshape(m // IMG_BINSIZE, IMG_BINSIZE, n // IMG_BINSIZE, IMG_BINSIZE)
    arr_binned = arr_re.sum(3).sum(1)
    arr = arr_binned // (arr_binned.max() / 255)

    arr = arr.astype(np.uint8)

    # Process the image to make fiducials stand out
    inverted = cv.threshold(arr, 0, 255, cv.THRESH_BINARY_INV)[1]
    nonoise = cv.fastNlMeansDenoising(inverted, None, h=100)
    thresholded2 = cv.threshold(nonoise, 128, 255, cv.THRESH_BINARY)[1]

    # Run two rounds of Hough transforms to identify the nested fiducials 
    circles1 = cv.HoughCircles(thresholded2,cv.HOUGH_GRADIENT,1,20,
                                param1=50,param2=20,minRadius=40,maxRadius=80)
    circles2 = cv.HoughCircles(thresholded2,cv.HOUGH_GRADIENT,1,20,
                                param1=50,param2=20,minRadius=15,maxRadius=30)

    # If we can't identify all of the circes from a set, be a bit more lenient on detection
    if circles1.shape[1] != 8 or circles2.shape[1] !=8:        
        circles1 = cv.HoughCircles(thresholded2,cv.HOUGH_GRADIENT,1,20,
                                    param1=50,param2=30,minRadius=30,maxRadius=70)
        
        
        circles2 = cv.HoughCircles(thresholded2,cv.HOUGH_GRADIENT,1,20,
                                    param1=50,param2=30,minRadius=10,maxRadius=30)
        if circles1.shape[1] != 8 or circles2.shape[1] !=8:
            return (f'{tile_id}', None, None)

    centers1 = circles1[0, :, :2]
    centroid1 = centers1.mean(0)
    centers2 = circles2[0, :, :2]
    centroid2 = centers2.mean(0)

    if not return_images:
        return (f'{tile_id}', None, {
            'circles1': circles1,
            'circles2': circles2,
            'centroid1': centroid1,
            'centroid2': centroid2
        })
        
    # Recreate images if they are wanted for QC
    cimg = cv.cvtColor(thresholded2, cv.COLOR_GRAY2BGR)
    circles = np.uint16(np.around(circles1))
    for i in circles[0,:]:
        cv.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
        cv.circle(cimg,(i[0],i[1]),2,(0,0,255),3)
    circles = np.uint16(np.around(circles2))
    for i in circles[0,:]:
        cv.circle(cimg,(i[0],i[1]),i[2],(255,0,0),2)
        cv.circle(cimg,(i[0],i[1]),2,(0,255,0),3)
    
    if circles1[0].shape[0] == 8 and circles2[0].shape[0] == 8:
        cv.line(cimg, (int(centroid1[0]) - 20, int(centroid1[1]) -20), (int(centroid1[0]) + 20, int(centroid1[1]) + 20), (0, 255, 0), 2)
        cv.line(cimg, (int(centroid2[0]) - 20, int(centroid2[1]) +20), (int(centroid2[0]) + 20, int(centroid2[1]) - 20), (255, 0, 0), 2)
    elif circles1[0].shape[0] == 8:
        cv.circle(cimg, (int(centroid1[0]), int(centroid1[1])), 4, (0, 255, 0))
    elif circles2[0].shape[0] == 8:
        cv.circle(cimg, (int(centroid2[0]), int(centroid2[1])), 4, (255, 0, 0))

    return (f'{tile_id}', {
        'raw': arr,
        'thresh1': inverted,
        'denoise': nonoise,
        'thresh2': thresholded2,
        'anno': cimg
    }, {
        'circles1': circles1,
        'circles2': circles2,
        'centroid1': centroid1,
        'centroid2': centroid2
    })
    

In [ ]:
tiles = os.listdir(TILE_DIR)

In [ ]:
tiles = ['_'.join(x.split('_')[:2]) for x in tiles if not 'subset' in x]

In [ ]:
all_tile_ids = []
for tile in tiles:
    all_tile_ids.append((TILE_DIR, tile))

# Multiprocesses tiles in parallel, adjust processes depending on the system configuration
with Pool(processes=72) as pool:
    results = pool.map(process_tile, all_tile_ids)

In [ ]:
# Did we miss any tiles? There are tiles where fiducial detection failed
for lane in range(1, 5):
    for surface in range(1, 3):
        for swath in range(1, 7):
            for tile in range(1, 79):
                final_tileid = f'{lane}_{surface}{swath}{tile:02}'
                if not os.path.exists(f'{TILE_DIR}/{final_tileid}_barcodes.pickle'):
                    print(f'No file for lane {lane} - {final_tileid}')

In [ ]:
ci_info = {}

In [ ]:
for x in results:
    ci_info[x[0]] = x[2]

Save the results to disk

In [ ]:
with open(f'{TILE_DIR}/circle_info.pickle', 'wb') as fh:
    pickle.dump(ci_info, fh)